In [1]:
import pandas as pd

In [2]:
# descobri o mes pelo numero da semana

from datetime import datetime, timedelta
 
def descobre_mes(ano, num_semana):    
    date = str(ano) + '-W' + str(num_semana)
    res = datetime.strptime(date + '-1', "%Y-W%W-%w")
    return res.strftime("%B")

In [3]:
def extrair_elementos(lista_str):
    try:
        lista = ast.literal_eval(lista_str)
        return ', '.join(lista)
    except (SyntaxError, ValueError):
        return None

songs + artistas

In [4]:
import ast

hit_songs = pd.read_csv('../songs_data/hit_songs/spotify_hits_dataset_complete.csv', delimiter='\t')
hit_songs = hit_songs.explode(['artist_id','artist_name']).reset_index(drop=True)
hit_songs['artist_id'] = hit_songs['artist_id'].apply(extrair_elementos)

artists = pd.read_csv('../songs_data/artists/spotify_artists_info_complete_reduced_genres.csv', delimiter='\t')

songs_and_artists = pd.merge(hit_songs, artists, on='artist_id')

agg_funcs = {
    'song_name': 'first',
    'artist_id': list,
    'artist_name': list, 
    'popularity_x': 'first',
    'explicit': 'first', 
    'song_type': 'first', 
    'track_number':'first', 
    'num_artists':'first',
    'num_available_markets':'first', 
    'release_date':'first', 
    'duration_ms':'first', 
    'key':'first', 
    'mode':'first',
    'time_signature':'first', 
    'acousticness':'first', 
    'danceability':'first', 
    'energy':'first',
    'instrumentalness':'first', 
    'liveness':'first', 
    'loudness':'first', 
    'speechiness':'first', 
    'valence':'first',
    'tempo':'first', 
    'name':'first', 
    'followers':'first', 
    'popularity_y':'mean', 
    'genres':list,#lambda x: list(set(x)),#list, 
    'image_url':'first'
}
songs_and_artists = songs_and_artists.groupby('song_id').agg(agg_funcs).reset_index()

change_name_columns ={
    'popularity_x': 'song_popularity',
    'name':'artist_name',  
    'popularity_y':'mean_artists_popularity', 
    'genres':'artists_genres'
}
songs_and_artists = songs_and_artists.rename(columns=change_name_columns)
songs_and_artists["artists_genres"].head()


0              [['sertanejo', 'brazilian funk']]
1    [['r&b', 'pop', 'dance pop', 'electropop']]
2                       [['indie rock', 'rock']]
3              [['indie pop', 'electro', 'pop']]
4                           [['rap', 'hip hop']]
Name: artists_genres, dtype: object

In [5]:
charts_brasil = pd.read_csv('../my_data/charts/brazil_charts.csv')

charts_brasil_songs = pd.merge(charts_brasil, songs_and_artists, on='song_id')
charts_brasil_songs['month'] = charts_brasil_songs.apply(lambda row: descobre_mes(row['year'], row['week']), axis=1)

charts_brasil_songs['genres'] = charts_brasil_songs['artists_genres'].apply(lambda x: list(pd.Series(x).explode().unique()))

colunas_de_analise = ['song_id', 'song_name_x','genres','artists_genres', 'year', 'month', 'artist', 'energy', 'danceability', 'streams','position']
charts_brasil_songs[colunas_de_analise].tail()

,song_id,song_name_x,genres,artists_genres,year,month,artist,energy,danceability,streams,position
17033,3pgO4N6Xkr8Ct0U9RgkrQh,Sem Rede - Ao Vivo,[['sertanejo']],[['sertanejo']],2019,December,Henrique & Juliano,0.566,0.543,1038976,101
17034,7vZqxuOvEwRHsGVaDykzpY,Porta Mala,[['sertanejo']],[['sertanejo']],2019,December,Os Parazim,0.904,0.807,834369,139
17035,3ZkTtSBN1wcSw35F8GDRmr,Chora Até Perder a Fala - Ao Vivo,[['sertanejo']],[['sertanejo']],2019,December,Gusttavo Lima,0.877,0.805,681995,185
17036,3dPvQLKl42dMYWV4LWRTLK,Tá Rocheda,[[]],[[]],2019,December,Os Barões Da Pisadinha,0.886,0.775,643682,193
17037,4pl0csAczZemDLdyNYOmxT,A Fila Anda - Ao Vivo,"[['pagode', 'sertanejo']]","[['pagode', 'sertanejo']]",2019,December,Thiaguinho,0.697,0.751,642627,195


In [6]:
data_reduced = charts_brasil_songs.groupby(['month', 'year']).agg({'energy': 'mean', 
                                                                     'danceability': 'mean',
                                                                     'streams': 'sum'}).reset_index()
data_reduced = data_reduced[['month', 'year', 'danceability', 'energy', 'streams']]
data_reduced.head()

,month,year,danceability,energy,streams
0,April,2017,0.681410,0.709404,258227772
1,April,2018,0.681420,0.693861,471285738
2,April,2019,0.681680,0.693038,467657469
3,August,2017,0.665500,0.709667,309530787
4,August,2018,0.678169,0.698484,402686263


In [7]:
import altair as alt

ordered_months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

dropdown_year = alt.binding_select(
    options=data_reduced['year'].unique().tolist(),
    name='Escolha o ano '
)
year_param = alt.param(
    value=2017,
    bind=dropdown_year
)

dropdown_song = alt.binding_select(
    options=['energy', 'danceability', 'streams'],
    name='Escolha musica '
)
ycol_param = alt.param(
    value='energy',
    bind=dropdown_song
)

grafico = alt.Chart(data_reduced,  width=800, height=300).mark_line().encode(
    alt.X('month', type='nominal', title='aaa', sort=ordered_months),
    alt.Y('y', type='quantitative', title=''),
    # color='Origin:N'
).transform_calculate(
    y=f'datum[{ycol_param.name}]'

).transform_filter(
    # year_param
    alt.FieldEqualPredicate(field='year', equal=year_param)

).add_params(
    ycol_param,
    year_param

)

circulos = grafico.mark_circle()
(grafico + circulos).properties(padding=20)

alt.LayerChart(...)

In [8]:
colunas_barras = ['position', 'year', 'month', 'streams', 'song_name_x']
barras = charts_brasil_songs[colunas_barras]
barras

,position,year,month,streams,song_name_x
0,1,2017,January,4792078,Deu Onda
1,1,2017,January,3477856,Deu Onda
2,1,2017,January,2649642,Deu Onda
3,2,2017,January,2105580,Deu Onda
4,4,2017,January,1931738,Deu Onda
...,...,...,...,...,...
17033,101,2019,December,1038976,Sem Rede - Ao Vivo
17034,139,2019,December,834369,Porta Mala
17035,185,2019,December,681995,Chora Até Perder a Fala - Ao Vivo
17036,193,2019,December,643682,Tá Rocheda
